Pedro Ivo Vasconcelos - TP

## **Capítulo 8**
1) Para responder isto é necessário verificar se os estados $x_{2}$ e $x_{3}$ são além de observáveis são também controláveis

Pegando a matriz C da questão 6 no capítulo 7:

$C = \begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0
\end{bmatrix}$

Fazendo a matriz de observabilidade a partir de A e C para representar o pH, é possível verificar que somente dois estados são observáveis.

$O_{bsv}$ = $\begin{bmatrix}
C \\
CA \\
CA^2 \\
\vdots \\
CA^{n-1}
\end{bmatrix}$

E analisando o problema de controlabilidade com a nova $O_{bsv}$, a controlabilidade pode ser verficada por:

$C_{trb}$\=\[B $O_{bsv}$ B  $O_{bsv}$<sup>2</sup>B ⋯ $O_{bsv}$<sup>n−1</sup>B\]

In [35]:
import numpy as np
from numpy.linalg import matrix_rank

# Definindo a matriz A e a matriz C
A = np.diag([0.031, 0.791, 0.791, 1.0, 1.0, 1.0, 1.0])
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],  # x2
    [0, 0, 1, 0, 0, 0, 0]   # x3
])

# Definindo a matriz B
B = np.array([
    [0.1545, 0.1545],
    [2.47e-4, -2.49e-4],
    [-4.56e-5, -4.15e-5],
    [-0.0303, 0],
    [0, 0],
    [0, -0.0303],
    [0, 0]
])

# Calculando a matriz de observabilidade O_bsv
n = A.shape[0]
obsv = C
for i in range(1, n):
    obsv = np.vstack((obsv, C @ np.linalg.matrix_power(A, i)))

# Construindo a matriz de controlabilidade usando O_bsv e B
ctrb = B
for i in range(1, n):
    ctrb = np.hstack((ctrb, np.linalg.matrix_power(A, i) @ B))

# Calculando o posto da matriz de controlabilidade usando a matriz de observabilidade
obsv_ctrb = obsv @ ctrb

# Calculando o posto da matriz obsv_ctrb
rank_of_controllability = matrix_rank(obsv_ctrb)
print("Posto da matriz de controlabilidade dos estados observáveis:", rank_of_controllability)

if rank_of_controllability == C.shape[0]:
    print("Os estados observáveis são controláveis.")
else:
    print("Os estados observáveis não são controláveis.")

Posto da matriz de controlabilidade dos estados observáveis: 2
Os estados observáveis são controláveis.


2. Para obter uma constante de tempo igual a metade do tempo

In [1]:
import numpy as np
from scipy.signal import place_poles

# Definindo A e B
A = np.array([[0.791, 0], [0, 0.791]])
B = np.array([[2.47e-4, -2.49e-4], [-4.56e-5, -4.15e-5]])

# Novos autovalores desejados
desired_poles = np.array([1.582, 1.582])

# Calculando a matriz K usando alocação de polos
K = place_poles(A, B, desired_poles).gain_matrix

# Verificando os autovalores do sistema em malha fechada
A_cl = A - B @ K
eigenvalues = np.linalg.eigvals(A_cl)
(K, eigenvalues)

(array([[-1519.40069151,  9116.40414906],
        [ 1669.51015742,  9043.18001935]]),
 array([1.582, 1.582]))